In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import json

In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key = api_key)

In [ ]:
completion = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role" : "system", "content" : "너는 귀여워. 사투리를 써"}, #시스템 캐릭터성 부여
        {"role" : "user", "content" : "오늘 야식 뭐 먹을래?"},#role, content 고정
        {"role" : "assistant", "content" : "나는 오늘 치킨이 땡기는데?"},
        {"role" : "user", "content" : "치킨이라니! 어디서 시킬까?"}
    ]
    
)

In [17]:
content = completion.choices[0].message.content
print(content)

음, 근처에 맛있는 치킨집 몇 군데 있지. 신선한 재료로 만드는 집이나 치밥이 맛있는 집 알아? 아니면 배달어플로 추천받아볼까?
